In [1]:
import numpy as np
import xarray as xr
import copy 

In [2]:
stepsize = 1
window = 15
start = int((window-1)/2)
ref_min = 1985
ref_max = 2014

In [3]:
# compute climatology
run = ['HIST']
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
    file = 'nGE_Tmax_'+n_run+'_dm_land_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax
        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    # ------------------ Load second set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/ens31_50/'
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe.nc'
    
    with xr.open_dataset(path+file) as data2:
        data2 = data2.tasmax
        #Get rid off 29th of februarys
        data2 = data2.sel(time=~((data2.time.dt.month == 2) & (data2.time.dt.day == 29)))
        
    #concatenate over ensemble dimension
    data = np.concatenate((data1.values,data2.values),axis=1)
    
    #flatten lon/lat
    data_flat = data.reshape(data.shape[0],data.shape[1],-1) 
    
    #calculate the climatology for the reference time
    if n_run == 'HIST':
        climatology = []
        r_steps = 365-window+1 #number of different running windows
        for r in range(r_steps):
            sel_days = []
            for y in range(ref_min,ref_max+1):
                data_sel = data_flat[(data1.time.dt.year==y)]
                data_sel = data_sel[r:r+window]
                sel_days.append(data_sel)
            
            sel_days_mean = np.array(sel_days).mean(axis=(0,1,2))
            climatology.append(sel_days_mean)
        climatology = np.array(climatology)

        climatology_complete = np.zeros((365,climatology.shape[1])) #fill values missing due to running window
        start = int((window-1)/2)
        climatology_complete[:start] = climatology[0]
        climatology_complete[-start:] = climatology[-1]
        climatology_complete[start:-start] = climatology

In [4]:
run = ['HIST','ssp119','ssp126','ssp245','ssp370','ssp585']

In [5]:
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
    file = 'nGE_Tmax_'+n_run+'_dm_land_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax
        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))

    #add ensemble dimension and reshape lon/lat to original format
    climatology_expanded = np.repeat(climatology_complete[:,np.newaxis,:],data1.shape[1],axis=1)
    climatology_expanded = climatology_expanded.reshape(-1,data1.shape[1],data1.shape[2],data1.shape[3])
        
    #calculate anomalies
    for y in np.unique(data1.time.dt.year):
        year_sel = data1[data1.time.dt.year==y].values
        anomalies1 = year_sel - climatology_expanded
        data1[data1.time.dt.year==y] = anomalies1


    # ------------------ Load second set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/ens31_50/'
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe.nc'
    
    with xr.open_dataset(path+file) as data2:
        data2 = data2.tasmax
        #Get rid off 29th of februarys
        data2 = data2.sel(time=~((data2.time.dt.month == 2) & (data2.time.dt.day == 29)))

    #add ensemble dimension and reshape lon/lat to original format
    climatology_expanded = np.repeat(climatology_complete[:,np.newaxis,:],data2.shape[1],axis=1)
    climatology_expanded = climatology_expanded.reshape(-1,data2.shape[1],data2.shape[2],data2.shape[3])
        
    #calculate anomalies
    for y in np.unique(data2.time.dt.year):
        year_sel = data2[data2.time.dt.year==y].values
        anomalies2 = year_sel - climatology_expanded
        data2[data2.time.dt.year==y] = anomalies2


# ------------------ save data -----------------------
    path = '/work/uo1075/u241308/data_python_PostDoc/MPI_GE_cmip6/t2max/'
    
    file = 'nGE_Tmax_'+n_run+'_dm_ens_0-30_land_europe_ens_mean_anomaly.nc'
    data1.to_netcdf(path+file)
    
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe_ens_mean_anomaly.nc'
    data2.to_netcdf(path+file)
    
    print(n_run + ' finished')

HIST finished
ssp119 finished
ssp126 finished
ssp245 finished
ssp370 finished
ssp585 finished
